In [1]:
import pandas as pd 
import numpy as np
import os
from pathlib import Path

In [2]:
onedrive_base = Path(os.environ.get('onedrive' , 'C:/Users/azona/OneDrive - Cerity Partners'))
process_base = onedrive_base / 'Portfolio Management Team' / '2 - Monthly AUM and Performance' / 'DEV'
input_path = onedrive_base / process_base

In [4]:
os.chdir(input_path)
file_name = 'Perf Individual Advisors.xlsx'
data = pd.read_excel(file_name)

In [5]:
data = data.drop(data[data['CP - Advisor'] == 'Total'].index)

### This next code block might not be needed if we get some other type of advisor ID in the script

In [6]:
column_name = data.columns[0]  # or use specific column name like 'Column A'

# Known category labels that indicate rows to skip
categories = ['Cash & Cash Equivalents', 'Fixed Income', 'Equities', 
              'Specialty', 'Private Markets']

# Create new column for Advisor name (as key)
advisor_counter = 0
advisor_names = []
current_advisor = None

for idx, value in enumerate(data[column_name]):
    # Convert to string and strip whitespace
    value_str = str(value).strip()
    
    # Check if this row is a category (not an advisor name)
    is_category = value_str in categories or pd.isna(value) or value_str == 'nan'
    
    if not is_category:
        # This is a new advisor
        advisor_counter += 1
        current_advisor = value_str
    
    # Assign current advisor name to this row
    advisor_names.append(current_advisor if current_advisor else None)

# Add the advisor name column to dataframe
data['Advisor_Name'] = advisor_names

In [8]:
data

,CP - Advisor,"Starting Adjusted Value (No Div, USD)","Adjusted Net Cash Flow (YTD, No Div, USD)","Adjusted Total Return (No Div, USD)","Adjusted Value (No Div, USD)","Adjusted TWR (Current Month, No Div, USD)","Adjusted TWR (YTD, No Div, USD)","1 Yr. Adjusted TWR (No Div, USD)","3 Yr. Annualized Adjusted TWR (No Div, USD)","Adjusted TWR (No Div, USD)",Advisor_Name
0,Adam B. Landau,1.186387e+09,60583958.864104,1.243680e+08,1.371339e+09,0.00126,0.100889,0.090751,0.096761,0.100889,Adam B. Landau
1,Cash & Cash Equivalents,8.236417e+07,13560826.68,4.018063e+06,9.994306e+07,0.003196,0.032561,0.035729,0.040736,0.032561,Adam B. Landau
2,Fixed Income,2.481896e+08,13649607.232088,1.498395e+07,2.768231e+08,0.002112,0.056843,0.05827,0.070223,0.056843,Adam B. Landau
3,Equities,4.700737e+08,27419710.260505,7.282135e+07,5.703148e+08,0.003323,0.15308,0.111199,0.149003,0.15308,Adam B. Landau
4,Specialty,1.573683e+08,-30063232.02,1.695153e+07,1.442566e+08,0.004112,0.119651,0.114626,0.074838,0.119651,Adam B. Landau
...,...,...,...,...,...,...,...,...,...,...,...
909,Cash & Cash Equivalents,0.000000e+00,99473.5,0.000000e+00,9.947350e+04,0,0,0,0,0,Michael O’Brien
910,Fixed Income,0.000000e+00,199736.74,5.000300e+02,2.002368e+05,0.002503,0.002503,0.002503,0.002503,0.002503,Michael O’Brien
911,Equities,0.000000e+00,1937766.13,7.609400e+04,2.013860e+06,0.039269,0.039269,0.039269,0.039269,0.039269,Michael O’Brien
912,"Nicole Joko, CFP®",0.000000e+00,0,0.000000e+00,0.000000e+00,-,-,-,-,-,"Nicole Joko, CFP®"


In [11]:
mapping_rules = (
    ('data["CP - Advisor"] == data["Advisor_Name"]', 'Starting Adjusted Value (No Div, USD)', 'Starting Value (12/31/2024)'),
    ('data["CP - Advisor"] == data["Advisor_Name"]', 'Adjusted Total Return (No Div, USD)', 'Total Return'),
    ('data["CP - Advisor"] == data["Advisor_Name"]', 'Adjusted Value (No Div, USD)', 'Ending Value (11/30/2025)'),
    ('data["CP - Advisor"] == data["Advisor_Name"]', 'Adjusted TWR (Current Month, No Div, USD)', 'Nov 2025 TWR'),
    ('data["CP - Advisor"] == data["Advisor_Name"]', 'Adjusted TWR (YTD, No Div, USD)', 'YTD TWR'),
    ('data["CP - Advisor"] == data["Advisor_Name"]', '1 Yr. Adjusted TWR (No Div, USD)', '1Y TWR'),
    ('data["CP - Advisor"] == data["Advisor_Name"]', '3 Yr. Annualized Adjusted TWR (No Div, USD)', '3Y TWR'),
    ('Equities', 'Adjusted TWR (YTD, No Div, USD)', 'EQ - YTD TWR'),
    ('Equities', '1 Yr. Adjusted TWR (No Div, USD)', 'EQ - 1Y TWR'),
    ('Equities', '3 Yr. Annualized Adjusted TWR (No Div, USD)', 'EQ - 3Y TWR'),
    ('Fixed Income', 'Adjusted TWR (YTD, No Div, USD)', 'FI - YTD TWR'),
    ('Fixed Income', '1 Yr. Adjusted TWR (No Div, USD)', 'FI - 1Y TWR'),
    ('Fixed Income', '3 Yr. Annualized Adjusted TWR (No Div, USD)', 'FI - 3Y TWR'),
    ('Cash & Cash Equivalents', 'Adjusted Value (No Div, USD)', 'Total - Cash'),
    ('Fixed Income', 'Adjusted Value (No Div, USD)', 'Total - FI'),
    ('Equities', 'Adjusted Value (No Div, USD)', 'Total - Equities'),
    ('Private Markets', 'Adjusted Value (No Div, USD)', 'Total - Private Markets')
)

In [12]:
all_results = []

for advisor in data['Advisor_Name'].dropna().unique():
    # Get all rows for this advisor
    advisor_data = data[data['Advisor_Name'] == advisor].copy()
    
    results = {'Advisor': advisor}
    
    for condition, input_column, output_column in mapping_rules:
        # Handle the special condition for advisor row
        if 'data["CP - Advisor"] == data["Advisor_Name"]' in condition:
            # This means we want the row where CP - Advisor matches the advisor name
            mask = advisor_data['CP - Advisor'] == advisor
        else:
            # This is a category match (Equities, Fixed Income, etc.)
            mask = advisor_data[column_name] == condition
        
        if mask.any() and input_column in advisor_data.columns:
            value = advisor_data.loc[mask, input_column].iloc[0]
            results[output_column] = value
        else:
            results[output_column] = None
    
    all_results.append(results)

# Create flattened DataFrame
output_data = pd.DataFrame(all_results)

### This will need to be merged with "2 - Monthly AUM and Performance -- Flatten out Performance Files" but we will want to filter out records where the advisor name is the same, and select the ones that come from the individual file... NOT THIS BIG COMBINED FILE

In [13]:
output_data

,Advisor,Starting Value (12/31/2024),Total Return,Ending Value (11/30/2025),Nov 2025 TWR,YTD TWR,1Y TWR,3Y TWR,EQ - YTD TWR,EQ - 1Y TWR,EQ - 3Y TWR,FI - YTD TWR,FI - 1Y TWR,FI - 3Y TWR,Total - Cash,Total - FI,Total - Equities,Total - Private Markets
0,Adam B. Landau,1.186387e+09,1.243680e+08,1.371339e+09,0.00126,0.100889,0.090751,0.096761,0.15308,0.111199,0.149003,0.056843,0.05827,0.070223,9.994306e+07,2.768231e+08,5.703148e+08,2.800016e+08
1,AJ Simoncelli,5.908571e+04,1.034662e+04,7.392951e+04,0.000864,0.17148,0.14403,0.166113,0.17557,0.147515,0.172154,None,None,None,3.886710e+03,NaN,7.004280e+04,NaN
2,Allen Kozel,8.496168e+07,1.108863e+07,1.001684e+08,0.008429,0.127064,0.101899,0.11225,0.172569,0.131569,0.156353,0.052697,0.040923,0.041804,1.073828e+07,1.606391e+07,6.415497e+07,8.748282e+06
3,"Andrejs Pilajevs, CFP®, MSFP",7.514487e+06,2.210819e+06,2.925348e+07,0.006917,0.145018,0.117221,0.157692,0.208559,0.163484,0.211574,0.061496,0.051662,0.068605,2.808167e+06,8.370307e+06,1.717015e+07,NaN
4,"Andrew Blumenthal, CFP®",4.011067e+08,4.514652e+07,4.501849e+08,0.00248,0.113123,0.095645,0.113758,0.163747,0.123479,0.167257,0.063283,0.057386,0.050336,3.526653e+07,5.013142e+07,2.285078e+08,7.244776e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,"Caroline Good, CFP®",9.952533e+05,3.617784e+05,5.489370e+06,0.009962,0.082922,0.054406,0.075287,0.176884,0.13995,0.163734,0.054393,0.042625,0.049603,3.523224e+05,2.277007e+06,2.858929e+06,NaN
161,"John R. Garris, CFP®",2.231261e+07,3.051137e+06,2.651127e+07,0.011421,0.131212,0.113479,0.120049,0.175505,0.148475,0.163864,0.055583,0.052437,0.047387,6.206211e+05,9.025296e+06,1.680717e+07,NaN
162,"Matthew J. Canner, CPWA®",2.295949e+08,3.718835e+07,2.662811e+08,0.009008,0.160025,0.128594,0.121008,0.201043,0.152484,0.149601,0.054051,0.040237,0.038419,1.461038e+07,4.207388e+07,1.924464e+08,1.202091e+07
163,Michael O’Brien,0.000000e+00,7.659403e+04,2.313570e+06,0.034899,0.034899,0.034899,0.034899,0.039269,0.039269,0.039269,0.002503,0.002503,0.002503,9.947350e+04,2.002368e+05,2.013860e+06,NaN


In [15]:
output_path = 'output_with_ids.xlsx'
output_data.to_excel(output_path, index=False)